In [ ]:
import igraph as ig

## 每个点的文档

因为我的图里存了每个节点每条边的名称，所以可以直接从igraph里获取

In [ ]:
def make_nne_docs(self):
    nne_docs = []
    self.g: ig.Graph
    for vid in range(self.g.vcount()):
        out_edges = self.g.vs[vid].out_edges()
        nne_fact_list = []
        this_name = self.g.vs[vid]["name"]
        for edge in out_edges:
            relation_name = edge['name']
            if relation_name in ['_synonym']:
                continue
            other_name = edge.target_vertex["name"]
            if edge['dir'] == 'f':
                nne_fact_list.append(f"{this_name} {relation_name} {other_name}.")
            elif edge['dir'] == 'b':
                nne_fact_list.append(f"{other_name} {relation_name} {this_name}.")
        nne_fact_list.sort()
        nne_fact = f"{this_name}\n" + " ".join(nne_fact_list)
        nne_docs.append(nne_fact)
        # nne_docs.append([self.g.vs[n]["name"] for n in neighbors if self.g.vs[n]["type"] == "passage"])
    return nne_docs

## 做retrieve Load进来

In [ ]:
with open(self.config.get_path("nne_docs_path"), "r") as f:
        self.nne_list = json.load(f)

    for nne_doc, entity_name in zip(self.nne_list, self.entity_list):
        assert nne_doc.startswith(entity_name), f"NNE_DOC {nne_doc[:20]} does not match entity {entity_name}"

## 计算相似度

In [ ]:
def get_nne_scores(self, query: Union[str, List[str]], 
                   instruction="Given a multihop query, retrieve the most helpful entity summary.", 
                   top_k=5) -> torch.Tensor:
    
    embedding = self.embeddings_dict["nne"]
    nne_scores = compute_similarity(
        model=self.model,
        query=query,
        indexed_embedding=embedding,
        instruction=instruction
    )
    topk_values, topk_indices = torch.topk(nne_scores, top_k)
    nne_scores = torch.zeros_like(nne_scores)
    nne_scores[topk_indices] = topk_values
    return nne_scores


后面就是和ner score一样地用